In [132]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error



In [133]:
#Gets the data from the scraper and deletes the index column
stats = pd.read_csv('total.csv')
del stats['Unnamed: 0']
raptor = pd.read_csv('modern_RAPTOR_by_player.csv')

In [134]:
raptor_clean = raptor[['player_name','raptor_total', 'season', 'mp']]

columns_to_rename = {'player_name': 'Player','raptor_total': 'Raptor'}
raptor_clean = raptor_clean.rename(columns = columns_to_rename)
raptor_clean[['mp','Raptor']] = raptor_clean[['mp','Raptor']].fillna(0)


In [135]:
#Gets Predidctor variables

columns_to_drop = ['SEASON',
 'PLAYER_ID',
 'FIRST_NAME',
 'LAST_NAME',
 'Player',
 'POSITION',
 'PER',
 'BPM',
 'Year',
 'Pick',
 'Age',
 'Pos',
 'STANDING_REACH_FT_IN']

predictors = (stats.drop(columns = columns_to_drop)).columns


In [136]:
def choose_year(year):
    
    #Training the model 
    train = stats[stats["Year"] < year]
    test = stats[stats['Year'] == year]
    reg = Ridge(alpha = 0.1)
    reg.fit(train[predictors], train['Pick'])

    #Getting predicitons in a dataframe
    predictions = reg.predict(test[predictors])
    predictions = pd.DataFrame(predictions, columns = ['Predictions'], index = test.index)
    combination = pd.concat([test[['Player', 'Pick']], predictions], axis = 1)
    
    #Getting Mean Square Error
    mse = mean_squared_error(combination['Pick'], combination['Predictions'])
    string = 'The mean square error for this draft is: ' + str(mse)
    
    #Displays rank which refers to each players draft position relative to one another in the given dataset as well as the predicted rank and difference
    #Not used cause clutter
    combination = combination.sort_values('Pick', ascending = True)
    combination['RK'] = list(range(1, combination.shape[0]+1))
    combination = combination.sort_values('Predictions', ascending = True)
    combination['Predicted_RK'] = list(range(1, combination.shape[0]+1))
    combination['Difference'] = combination['Predicted_RK'] - combination['RK']

    #Gets a clutter free df 'clean' which neatly displays information
    important_columns = ['Player',  'Pick',  'Predictions']
    clean = combination.loc[:,important_columns]
    clean['Predictions'] = round(clean['Predictions'], 0)
    clean['Predictions'] = clean['Predictions'].astype(int)
    clean['Difference'] = combination['Pick'] - clean['Predictions'] 
    
    #Raptor, next step is use outer merge to get players with 0 min play
    final = pd.merge(clean, raptor_clean[raptor_clean['season'] == 2021], on='Player', how = 'inner')
    final['Score'] = (final['Difference'] * final['mp'])
    score = sum(final['Score']) - sum(final['mp'])
    return final, score

    

In [137]:
#year = input("Enter the draft class you would like to predict: ")


In [138]:
#choose_year(int(year))

In [139]:
choose_year(int(2017))[0]

,Player,Pick,Predictions,Difference,Raptor,season,mp,Score
0,Josh Hart,30,17,13,0.789852,2021,1349,17537
1,Josh Hart,30,17,13,0.789852,2021,1349,17537
2,Semi Ojeleye,37,18,19,-1.603980,2021,962,18278
3,Donovan Mitchell,13,20,-7,3.841444,2021,2117,-14819
4,Thomas Bryant,42,21,21,-0.953888,2021,271,5691
5,Justin Patton,16,23,-7,-6.197479,2021,247,-1729
6,Jordan Bell,38,23,15,-8.794843,2021,82,1230
7,Jordan Bell,38,24,14,-8.794843,2021,82,1148
8,Kyle Kuzma,27,24,3,-3.083373,2021,2083,6249
9,Dillon Brooks,45,26,19,0.840660,2021,2172,41268


In [128]:
choose_year(int(2017))[1]

73327